### Persistence in Langgraph
This project demonstrates how to implement persistence in LangGraph, ensuring that workflows, states, and intermediate results are stored and retrievable across sessions. By adding persistence, the system maintains continuity, enabling workflows to resume from saved states instead of restarting, which is essential for building reliable, production-ready AI applications.

In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# InMemorySaver is a simple in-memory checkpoint saver
# InMemorySaver is used to save checkpoints in RAM
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()
llm = ChatOpenAI()

In [3]:
# Create a state
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [4]:
# define nodes

def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [5]:
# create graph with memory persistence

graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

# Compile the graph with an in-memory checkpointer
checkpointer = InMemorySaver()
workflow = graph.compile(checkpointer=checkpointer)

In [6]:
# creating thread with Persistence
# thread_id is used to identify the thread

config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!",
 'explanation': 'This joke is a play on words, using the idea of a pizza seeking therapy as if it were a person with emotional struggles. The punchline - "it had too many toppings and couldn\'t handle the pressure" - humorously suggests that the pizza was overwhelmed by the weight of all the toppings placed on it, causing it to seek professional help. The joke is meant to be light-hearted and silly, making light of the idea that a pizza could have psychological issues.'}

In [7]:
# workflow.get_state will return the current state (final) of the workflow

workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the idea of a pizza seeking therapy as if it were a person with emotional struggles. The punchline - "it had too many toppings and couldn\'t handle the pressure" - humorously suggests that the pizza was overwhelmed by the weight of all the toppings placed on it, causing it to seek professional help. The joke is meant to be light-hearted and silly, making light of the idea that a pizza could have psychological issues.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-2724-6c98-8002-209a37956ac9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:38:28.655819+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-1c4d-62db-8001-797bec3cdb83'}}, tas

In [8]:
# to check intermediate states, we can use get_state_history
# 4 history states will be there
# 1. initial state with topic
# 2. after generate_joke node
# 3. after generate_explanation node
# 4. final state

list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza go to the therapist? Because it had too many toppings and couldn't handle the pressure!", 'explanation': 'This joke is a play on words, using the idea of a pizza seeking therapy as if it were a person with emotional struggles. The punchline - "it had too many toppings and couldn\'t handle the pressure" - humorously suggests that the pizza was overwhelmed by the weight of all the toppings placed on it, causing it to seek professional help. The joke is meant to be light-hearted and silly, making light of the idea that a pizza could have psychological issues.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-2724-6c98-8002-209a37956ac9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:38:28.655819+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-1c4d-62db-8001-797bec3cdb83'}}, ta

In [9]:
# with a different thread_id, we can create a new thread

config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the fusilli feel left out of the pasta party? Because it couldn't find a spiral partner to dance with!",
 'explanation': "This joke plays on the shape of fusilli pasta, which is spiral-shaped. In the context of a pasta party, where different types of pasta are coming together to have fun, the fusilli feels left out because it couldn't find another spiral-shaped pasta to dance with. The humor comes from the personification of the pasta and the idea of pasta dancing together at a party."}

In [10]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the fusilli feel left out of the pasta party? Because it couldn't find a spiral partner to dance with!", 'explanation': "This joke plays on the shape of fusilli pasta, which is spiral-shaped. In the context of a pasta party, where different types of pasta are coming together to have fun, the fusilli feels left out because it couldn't find another spiral-shaped pasta to dance with. The humor comes from the personification of the pasta and the idea of pasta dancing together at a party."}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-38b9-6619-8002-019fc5e8d792'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:38:30.499251+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-2fea-6c07-8001-48f3dfcfb19a'}}, tasks=(), interrupts=())

In [11]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the fusilli feel left out of the pasta party? Because it couldn't find a spiral partner to dance with!", 'explanation': "This joke plays on the shape of fusilli pasta, which is spiral-shaped. In the context of a pasta party, where different types of pasta are coming together to have fun, the fusilli feels left out because it couldn't find another spiral-shaped pasta to dance with. The humor comes from the personification of the pasta and the idea of pasta dancing together at a party."}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-38b9-6619-8002-019fc5e8d792'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:38:30.499251+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-2fea-6c07-8001-48f3dfcfb19a'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': "Why did th

### Time Travel

Sometime it is helpful in debugging, we can replay graph from specific point.

In [13]:
# add thread_id and checkpoint_id to resume from a specific checkpoint
# copy thread id from above cell and checkpoint id from get_state_history

workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f09f8cb-278c-6131-8000-9315f6451c4a"}})

StateSnapshot(values={'topic': 'pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f09f8cb-278c-6131-8000-9315f6451c4a'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-10-02T12:38:28.698126+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-2789-6832-bfff-3853689c3fea'}}, tasks=(PregelTask(id='99ddf181-877a-56eb-0d86-bd5873e19951', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': "Why did the fusilli feel left out of the pasta party? Because it couldn't find a spiral partner to dance with!"}),), interrupts=())

In [14]:
# re-run to show time travel resume

workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f09f8cb-278c-6131-8000-9315f6451c4a"}})

{'topic': 'pasta',
 'joke': "Why did the pasta go to the party alone? Because it couldn't find a date sauce!",
 'explanation': 'This joke is a play on words. "Date sauce" sounds like "mate sauce," which refers to a partner or date for the party. The pasta couldn\'t find a "date sauce" because it couldn\'t find a date to accompany it to the party. The humor comes from the pun on the word "sauce" and the clever wordplay.'}

In [15]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the pasta go to the party alone? Because it couldn't find a date sauce!", 'explanation': 'This joke is a play on words. "Date sauce" sounds like "mate sauce," which refers to a partner or date for the party. The pasta couldn\'t find a "date sauce" because it couldn\'t find a date to accompany it to the party. The humor comes from the pun on the word "sauce" and the clever wordplay.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cc-b67a-6af2-8002-8a5495f0185d'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:39:10.529166+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cc-abbc-684b-8001-ce3f0e3ba985'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': "Why did the pasta go to the party alone? Because it couldn't find a date sauce!"}, next=('generate_explanation',),

#### Updating State

In [17]:
# we need to update the topic from state 2 from bottom because we want to generate a different joke
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f09f8cb-278c-6131-8000-9315f6451c4a", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09f8d0-1171-66df-8001-9a1c7f2404f9'}}

In [18]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d0-1171-66df-8001-9a1c7f2404f9'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-10-02T12:40:40.598068+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cb-278c-6131-8000-9315f6451c4a'}}, tasks=(PregelTask(id='c7e6642b-db2f-3872-41c9-0210782e3883', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8cf-c853-6b9e-8000-f47d2800d9f4'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-10-02T12:40:32.931308+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id

In [19]:
# make sure copy checkpoint_id from above cell, because we want to invoke from last checkpoint (Samosa joke)
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f09f8d0-1171-66df-8001-9a1c7f2404f9"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to the party? Because it wanted to be a party appetizer!',
 'explanation': 'This joke plays on the fact that samosas are a popular party appetizer often served at events and gatherings. The punchline of the joke is a play on words, as the samosa is going to the party not as a guest, but rather as an appetizer to be served at the party. The joke is light-hearted and humorous because it personifies the samosa as having its own desire to be a part of the party experience.'}

In [20]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to the party? Because it wanted to be a party appetizer!', 'explanation': 'This joke plays on the fact that samosas are a popular party appetizer often served at events and gatherings. The punchline of the joke is a play on words, as the samosa is going to the party not as a guest, but rather as an appetizer to be served at the party. The joke is light-hearted and humorous because it personifies the samosa as having its own desire to be a part of the party experience.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d0-fcfb-6be6-8003-156d954569dc'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-10-02T12:41:05.296250+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d0-f14e-69f1-8002-d17770f3f8b6'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did th

### Fault Tolerance

In this section, interrupt keyboard at step2 and then run the next cell by passing None. It will resume from last step.

In [21]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [22]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [23]:
# 2. Define nodes
def step_1(state: CrashState) -> CrashState:
    print("Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(10)
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("Step 3 executed")
    return {"done": True}

In [24]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [25]:
try:
    print("Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("Kernel manually interrupted (crash simulated).")

Running graph: Please manually interrupt during Step 2...
Step 1 executed
Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
Kernel manually interrupted (crash simulated).


In [26]:
graph.get_state({"configurable": {"thread_id": "thread-1"}})

StateSnapshot(values={'input': 'start', 'step1': 'done'}, next=('step_2',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d1-63d6-6bb9-8001-22e6e05d4909'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-10-02T12:41:16.081433+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d1-63d3-6c1e-8000-5737d93d63fd'}}, tasks=(PregelTask(id='7584f88a-5f53-176c-778d-eb8982c37ec5', name='step_2', path=('__pregel_pull', 'step_2'), error=None, interrupts=(), state=None, result=None),), interrupts=())

In [27]:
# Re-run to show fault-tolerant resume
# pass None as input to resume from last checkpoint
print("Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("Final State:", final_state)

Re-running the graph to demonstrate fault tolerance...
Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
Step 3 executed
Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [28]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d2-410c-69cb-8003-cbeedb61b383'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-10-02T12:41:39.277028+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d2-4109-6609-8002-b8556dc6e681'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d2-4109-6609-8002-b8556dc6e681'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-10-02T12:41:39.275695+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f09f8d1-63d6-6bb9-8001-22e6e05d4909'}}, tasks=(PregelTask(id='9d7f585d-12a6-8a36-5b12-0063f60c027